In [18]:
import json
import os
import shutil
import subprocess
import tempfile

metadata = '/home/joaquin/projects/methylation/data/commonData/ids_data_allReplicates_methylation.json'
basePathDataFolder = '/home/joaquin/projects/methylation/data'
narrowPeakLocationFolders = ['tfs_rep_1','tfs_rep_3_input_from_rep_2', 'tfs_rep_2', 'tfs_rep_4']
# Opening JSON file

results_folder_path = '/home/joaquin/projects/methylation/data/loops_subproject_jose/allNarrowpeaks'
experiments_dic = {}
input_narrowpeaks_folders = ['data_loops_fl_1','data_loops_fl_2']
input_narrowpeaks_subfolders = ['MYC2FL/loopPIF4','MYC2FL/loopPIF5','MYC2FL/WT']

# parse the experiments file to get the id of each exeriment and the path to that experiment
for experiment in input_narrowpeaks_subfolders:
    experiments_dic[experiment] = []
    
    for parent_folder in input_narrowpeaks_folders:
        path = os.path.join(basePathDataFolder,parent_folder,experiment)
        filesInFolder = os.listdir(path)
    
        narrowpeakFileOriginalPath = os.path.join(
            path,'GEMout','GEMout.GEM_events.narrowPeak'
        )
        narrowpeakFileDestinationPath = os.path.join(
        results_folder_path,experiment.reaplace('/','_')+'_replicate'+parent_folder[-1]+'.narrowPeak'
            )
        experiments_dic[experiment].append(narrowpeakFileDestinationPath)
        #shutil.copy(narrowpeakFileOriginalPath,narrowpeakFileDestinationPath)
        #print(narrowpeakFileOriginalPath,narrowpeakFileDestinationPath)



In [4]:
from statistics import mean
import time
import glob
working_folder = '/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input'
destinationFilePath = 'allThePossiblePeaksnine_col0input.bed'
dictOfCommonNPEvents = {}
commonfile = []
spaceValue = 20
files = glob.glob(working_folder+'/*/*.GEM_events.narrowPeak')

def saveFile(dictOfCommonNPEvents, destinationFilePath):
    counter = 0
    with open(destinationFilePath, 'a+') as destFile:
        for chrm in [1,2,3,4,5]:
            for coordinates in dictOfCommonNPEvents[chrm]:
                counter +=1
                destFile.write('chr{}\t{}\t{}\t{}:{}\n'.format(chrm,coordinates[0],coordinates[1],chrm,coordinates[2]))
    print(counter)


for file in files:
    if '.GEM_events.narrowPeak' in file and '_1' not in file and '_2' not in file:
        # time.sleep(1)
        print(file)
        with open(file, 'r') as NPEvents:
            for line in NPEvents:
                chrAndCoor = list(map(lambda x: int(x), line.strip().split('\t')[3].split(':')))
                dictOfCommonNPEvents.setdefault(chrAndCoor[0], [])
                dictOfCommonNPEvents[chrAndCoor[0]].append(chrAndCoor[1])
for chrm in dictOfCommonNPEvents:
    sortEachChr = sorted(dictOfCommonNPEvents[chrm])
    dictOfCommonNPEvents[chrm] = sortEachChr

    m = [[dictOfCommonNPEvents[chrm][0]]]
    
    for x in dictOfCommonNPEvents[chrm][1:]:

            if x - m[-1][-1] < spaceValue:
                m[-1].append(x)
            else:
                meanValPeak = int(mean(m[-1]))
                m.pop()
                m.append([meanValPeak-100,meanValPeak+101, meanValPeak])
                m.append([x])

    meanValPeak = int(mean(m[-1]))
    m.pop()
    m.append([meanValPeak-100,meanValPeak+101, meanValPeak])
    dictOfCommonNPEvents[chrm] = m
saveFile(dictOfCommonNPEvents, destinationFilePath)


/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP8-col0/IP8-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP3-col0/IP3-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP5-col0/IP5-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP10-col0/IP10-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP11-col0/IP11-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP1-col0/IP1-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP14-col0/IP14-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP12-col0/IP12-col0.GEM_events.narrowPeak
/home/joaquin/projects/methylation/data/data_reciprocal_gem/col0input/IP7-col0/IP7-col0.GEM_events.narrowPeak
/h

In [11]:
import json
import os
import shutil
import subprocess
import tempfile
import glob
import pandas as pd
from functools import reduce
import re
import pandas as pd
import time

# reduce new function https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

commonpeaksfile = 'allThePossiblePeaksnine_col0input.bed'

basePathDataFolder = '/home/joaquin/projects/methylation/data'
def performIntersect(folder, intersectFile):
    
    sortedBamFile = glob.glob(f'{folder}/*orted.bam')
    
    if len(sortedBamFile) != 1:
        return print(folder, ' has a problem selecting File')
    else:
        sortedBamFile = sortedBamFile[0]
    
    outputFile = intersectFile.strip().split('/')[-1][:-4]+'_'+sortedBamFile.split('/')[-1][:-10]+'.bed'
    outputFilePath = os.path.join(folder,outputFile)


    print(outputFilePath)
    
    if os.path.isfile(outputFilePath):
        os.remove(outputFilePath)
    if not os.path.isfile(outputFilePath):
        subprocess.call(
            'samtools' + ' view -q1 -b ' + sortedBamFile + ' | ' +
            'bedtools' + ' intersect -abam stdin -b ' + intersectFile + ' -bed -wb -f 0.5 ' +
            '> ' + outputFilePath , shell=True
        )
    else:
        print(outputFile, ' is already done')
    
    totalForBox = {}
    with open(outputFilePath, 'r') as intersectOut:
        intersectDf = pd.read_csv(
            intersectOut, sep='\t', usecols=[3, 12, 13, 14, 15],
            names=['intersected', 'chr', 'start', 'end', 'boxname'],
        )
        for index, ip in intersectDf.iterrows():
            intersectOcurrence = str(ip.intersected.split('/')[0])
            box = ','.join([str(ip.chr), str(ip.start), str(ip.end), ip.boxname])
            if box in totalForBox:
                totalForBox[box].add(intersectOcurrence)
            else:
                totalForBox[box] = {intersectOcurrence}

        for box in totalForBox:
            boxlen = len(totalForBox[box])
            totalForBox[box] = boxlen

        with open(outputFilePath[:-4] + '_boxtotals.csv', 'w') as elcsv:
            elcsv.write('chr,start,end,boxname,{}\n'.format(sortedBamFile.split('/')[-1][:-10]))
            for name, recount in totalForBox.items():
                elcsv.write('{},{}\n'.format(name, recount))

listoffolders = glob.glob('/home/joaquin/projects/methylation/data/data_arabidopsis_ecotypes_IP/*/*/*')
# parse the experiments file to get the id of each exeriment and the path to that experiment
for experiment in listoffolders:
    for parent_folder in input_narrowpeaks_folders:
        print(experiment)
        performIntersect(experiment, commonpeaksfile)